In [ ]:
import itertools
import time
from threading import Thread, Event

def spin(msg: str, done: Event)->None:
    """
    runs in a separate thread
    done: instance of threading.Event
    """
    for char in itertools.cycle(r'\|/-'): #infinite loop
        status = f'\r{char} {msg}'
        print(status, end='', flush=True)
        if done.wait(.1): #return True when event is set by another thread
            break
        blanks = ''*len(status)
        print(f'\r{blanks}\r', end='')

def slow()->int:
    time.sleep(3) #blocks calling thread but releases the GIL
    return 42



In [ ]:
def supervisor() -> int:
    done = Event()
    spinner = Thread(target=spin, args=('thinking!', done))
    print(f'spinner object: {spinner}')
    spinner.start()
    result = slow()
    done.set()
    spinner.join()
    return result

def main() -> None:
    result = supervisor()
    print(f'result: {result}')

if __name__ == '__main__':
    main()

In [ ]:
import itertools
import time
from multiprocessing import Process, Event #event is a function not a class like threading.Event
from multiprocessing import synchronize

def spin(msg:str, done: synchronize.Event)->None:
    for char in itertools.cycle(r'\|/-'): #infinite loop
        status = f'\r{char} {msg}'
        print(status, end='', flush=True)
        if done.wait(.1): #return True when event is set by another thread
            break
        blanks = ''*len(status)
        print(f'\r{blanks}\r', end='')

def slow()->int:
    time.sleep(3) #blocks calling thread but releases the GIL
    return 42


def supervisor()->int:
    done = Event()
    spinner = Process(target=spin, args=('thinking!', done))
    print(f'spinner object: {spinner}')
    spinner.start()
    result = slow()
    spinner.join()
    return result

In [ ]:
import asyncio

#main stays blocked until supervisor returns
def main()->None:
    result = asyncio.run(supervisor()) #start event loop to drive coroutine
    print(f'result: {result}')

async def supervisor()->int: #native coroutines
    spinner = asyncio.create_task(spin('thinking!'))
    print(f'spinner object: {spinner}')
    result = await slow()
    spinner.cancel()
    return result

if __name__ == '__main__':
    main()


In [ ]:
import asyncio
import itertools

#asyncio has only one flow of execution

async def spin(msg: str)->None:
    for char in itertools.cycle(r'\|/-'):
        status = f'\r{char} {msg}'
        print(status, end='', flush=True)
        try:
            await asyncio.sleep(.1) #pause wo blocking other coroutines
        except asyncio.CancelledError:
            break
        blanks = ''*len(status)
        print(f'\r{blanks}\r', end='')

async def slow() -> int:
    await asyncio.sleep(3)
    return 42

In [ ]:
import math
from math import isqrt

def is_prime(n:int)->bool:
    if n < 2:
        return False
    if n == 2:
        return True
    if n % 2 == 0:
        return False
    root = math.isqrt(n)
    for i in range(3, root+1, 2):
        if n % i == 0:
            return False
        return True

In [12]:
#make is prime a coroutine to keep spinner alive

async def is_prime(n):
    if n < 2:
        return False
    if n == 2:
        return True
    if n % 2 == 0:
        return False
    root = isqrt(n)
    for i in range(3, root+1, 2):
        if n % i == 0:
            return False
        if i % 100000 == 0:
            await asyncio.sleep(0)
    return True
#still slow

In [ ]:
#Homegrown process pool

